# Hyper Parameter Optimization Techniques Contd.

## 3. Automated Hyperparameter Tuning

    Automated Hyperparameter Tuning can be done by using techniques such as

- Bayesian Optimization
- Gradient Descent
- Evolutionary Algorithms

## 3.1 Bayesian Optimization

    1. Bayesian optimization uses probability to find the minimum of a function. 
    2. The final aim is to find the input value to a function which can gives us the lowest possible output value.
    3. It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. 
    4. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

- Objective Function = 
    `defines the loss function to minimize.`
- Domain Space = 
    `defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).`
- Optimization Algorithm = 
    `defines the search algorithm to use to select the best input values to use in each new iteration.`

In [9]:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35,0,33.6,0.627,50,1
1,1,85.0,66,29,0,26.6,0.351,31,0
2,8,183.0,64,0,0,23.3,0.672,32,1
3,1,89.0,66,23,94,28.1,0.167,21,0
4,0,137.0,40,35,168,43.1,2.288,33,1


In [11]:
#### Independent And Dependent features
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [12]:
#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [2]:
!pip install hyperopt

In [3]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [4]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }


## hp.choice to select between a list
## hp.quniform to select between a range of no having saome equal
## hp.uniform to slecect btween floating no

In [5]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x2a16c279a88>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x2a16c279ec8>,
 'max_features': <hyperopt.pyll.base.Apply at 0x2a16c282488>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x2a16c282848>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x2a16c282c08>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x2a16c2844c8>}

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [7]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [13]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|███████████████████████████████████████████████| 80/80 [11:16<00:00,  8.46s/trial, best loss: -0.7622684259629482]


{'criterion': 0,
 'max_depth': 90.0,
 'max_features': 2,
 'min_samples_leaf': 0.014039745879919538,
 'min_samples_split': 0.10785280990155477,
 'n_estimators': 4}

In [14]:
## Since the above result in in key val pairs the below is done to get the vlue from key val

crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

entropy
log2
1200


In [15]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)

print('---------------------- confusion_matrix -------------------------')
print(confusion_matrix(y_test,predictionforest))

print('---------------------- accuracy_score -------------------------')
print(accuracy_score(y_test,predictionforest))

print('---------------------- classification_report -------------------------')
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

---------------------- confusion_matrix -------------------------
[[97 10]
 [22 25]]
---------------------- accuracy_score -------------------------
0.7922077922077922
---------------------- classification_report -------------------------
              precision    recall  f1-score   support

           0       0.82      0.91      0.86       107
           1       0.71      0.53      0.61        47

    accuracy                           0.79       154
   macro avg       0.76      0.72      0.73       154
weighted avg       0.78      0.79      0.78       154



In [ ]:
a